<h1>Importation des packages et des fonctions</h1>

In [1]:
import sys
sys.path.append('../../')

from fonctionsUtiles import functions as f
from fonctionsUtiles import architecture

import matplotlib.pyplot as plt
import numpy as np
import cv2
from sklearn.model_selection import train_test_split

import argparse
import warnings
warnings.filterwarnings("ignore")

__init__


In [2]:
import keras
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

from tensorflow.keras import layers
from tensorflow.python.keras.applications import ResNet50
from tensorflow.python.keras.applications import MobileNetV2
from tensorflow.python.keras.models import Sequential, Model
from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dropout

Using TensorFlow backend.


<h1>Chargement de toutes les données</h1>

In [3]:
# Get training data from the Axionable track
X_axio = np.load('../../Datasets/axionable_data/X_train_axio.npy')
Y_axio = np.load('../../Datasets/axionable_data/Y_train_axio.npy')
print('Axionable data Loaded. Shape = ', np.shape(X_axio))

# Get training data from IronCar track
# New track - Double chicane
X_chicane = np.load('../../Datasets/ironcar_data/new_track/x_chicane.npy')
Y_chicane = np.load('../../Datasets/ironcar_data/new_track/y_chicane.npy')
print('Ironcar new track chicane Loaded. Shape = ', np.shape(X_chicane))

# Old track - Balanced dataset
X_iron = np.load('../../Datasets/ironcar_data/old_track/balanced_iron_X.npy')
Y_iron = np.load('../../Datasets/ironcar_data/old_track/balanced_iron_Y.npy')
print('Ironcar old track data Loaded. Shape = ', np.shape(X_iron))

Axionable data Loaded. Shape =  (26449, 90, 250, 3)
Ironcar new track chicane Loaded. Shape =  (1519, 90, 250, 3)
Ironcar old track data Loaded. Shape =  (16028, 90, 250, 3)


<h1>Resizing des données</h1>

In [4]:
def resizing(array):
    masterX = list()
    for arr in array:
        masterX.append(cv2.resize(arr, (224,224)))
    return np.array(masterX)

X_axio_resized = resizing(X_axio)
X_chicane_resized = resizing(X_chicane)
X_iron_resized = resizing(X_iron)

print('Axionable data new size. Shape = ', np.shape(X_axio_resized))
print('Ironcar new track chicane new size. Shape = ', np.shape(X_chicane_resized))
print('Ironcar old track data new size. Shape = ', np.shape(X_iron_resized))

Axionable data new size. Shape =  (26449, 224, 224, 3)
Ironcar new track chicane new size. Shape =  (1519, 224, 224, 3)
Ironcar old track data new size. Shape =  (16028, 224, 224, 3)


In [ ]:
# X_axio_resized = X_axio
# X_chicane_resized = X_chicane
# X_iron_resized = X_iron

# print('Axionable data new size. Shape = ', np.shape(X_axio_resized))
# print('Ironcar new track chicane new size. Shape = ', np.shape(X_chicane_resized))
# print('Ironcar old track data new size. Shape = ', np.shape(X_iron_resized))

<h1>Séparation des données en Train, Test et Validation</h1>

In [5]:
# Dictionnaire qui stock quelques variables importantes
args = {"augmentation": True,
        "train_split": 0.8,
        "val_split":0.2,
        "test_split":0.2,
        "early_stop":True,
        "patience":2}

In [6]:
# Concatenation de toutes les données chargées
X = np.concatenate((X_axio_resized, X_chicane_resized, X_iron_resized))
Y = np.concatenate((Y_axio, Y_chicane, Y_iron))
print('All data loaded and concatenated. Shape = ', np.shape(X))

# Suppression des variables tmp qui contiennent les images
del X_axio, Y_axio, X_chicane, Y_chicane, X_iron, Y_iron, X_axio_resized, X_chicane_resized, X_iron_resized

All data loaded and concatenated. Shape =  (43996, 224, 224, 3)


In [7]:
# Réparatition du jeu de données en train test 80 - 20 %
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=12)

In [8]:
# Répartition du jeu d'entrainement en jeu d'entrainement et en jeu de validation
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=12)

In [9]:
print('All training data loaded and augmented. Shape = ', np.shape(X_train))
print('All validation data loaded and augmented. Shape = ', np.shape(X_val))
print('All testing data loaded and augmented. Shape = ', np.shape(X_test))

All training data loaded and augmented. Shape =  (28156, 224, 224, 3)
All validation data loaded and augmented. Shape =  (7040, 224, 224, 3)
All testing data loaded and augmented. Shape =  (8800, 224, 224, 3)


In [10]:
# Suppression des variables tmp qui contiennent les images
del X,Y

<h1>Augmentation des données</h1>

In [11]:
# Data augmentation of the dataset / Adjust the proportion of each transformation you want to apply.
if args['augmentation']:
    print('Augmenting data... Wait...')
    # Data augmentation 25% of random brightness.
    X_bright, Y_bright = f.generate_brightness(X_train, Y_train, proportion=0.001)
    # Data augmentation 25% of night effect.
    X_night, Y_night = f.generate_night_effect(X_train, Y_train, proportion=0.001)
    # Data augmentation 25% of horizontal flipping.
    X_flip, Y_flip = f.generate_horizontal_flip(X_train, Y_train, proportion=0.001)
    # Data augmentation 25% of random shadows.
    X_shadow, Y_shadow = f.generate_random_shadows(X_train, Y_train, proportion=0.001)
    # Data augmentation 25% of chained tranformations (bright + shadows + flip).
    X_chain, Y_chain = f.generate_chained_transformations(X_train, Y_train, proportion=0.001)

    # Concatenating Axionable dataset with the transformations.
    X_train = np.concatenate((X_train, X_bright, X_night,
                                X_flip, X_shadow, X_chain))

    Y_train = np.concatenate((Y_train, Y_bright, Y_night, 
                                Y_flip, Y_shadow, Y_chain)).astype('float32')

    print('Axionable data after augmentation. Shape = ', np.shape(X_train))

  0%|          | 0/28 [00:00<?, ?it/s]

Augmenting data... Wait...


100%|██████████| 28/28 [00:00<00:00, 295.70it/s]


Axionable data after augmentation. Shape =  (28296, 224, 224, 3)


In [12]:
# Suppression des variables tmp qui contiennent les images
del X_bright, Y_bright, X_night, Y_night, X_flip, Y_flip, X_shadow, Y_shadow, X_chain, Y_chain

<h1>Hyperparamètres</h1>

In [13]:
num_classes = 5
IMAGE_RES = 224
BATCH_SIZE = 400
epochs = 4

<h1>Construction du modèle</h1>

In [14]:
# Initialisation du modèle : on télécharge le modèle MobilNetV2 et on rajoute la dernière couche
# de classification propre à notre problème
my_new_model = Sequential()
my_new_model.add(MobileNetV2(include_top=False, pooling='avg', weights="imagenet"))
# my_new_model.add(ResNet50(include_top=False, pooling='avg', weights="imagenet"))
my_new_model.add(Flatten(name='flattened'))
my_new_model.add(Dense(128, activation='relu'))
my_new_model.add(Dropout(0.2))
my_new_model.add(Dense(64, activation='relu'))
my_new_model.add(Dropout(0.2))
my_new_model.add(Dense(num_classes, activation='softmax'))

# On indique que les couches avant la couche de classification ne doivent pas être entrainées
my_new_model.layers[0].trainable = False


# base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# l = Flatten()(base_model.output)
# predictions = Dense(num_classes, activation='softmax')(l)
# model = Model(inputs=base_model.input, outputs=predictions)

In [15]:
# On vérfie la structure du modèle
# my_new_model.summary()
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
______________________________________________________________________________________________

In [17]:
# for i, l in enumerate(my_new_model.layers) : 
#     print(i,l.output)
print(len(model.layers))

157


In [18]:
# On définit la fonction d'optimisation du modèle
# my_new_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [19]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
# Paramètre de réduction du learning rate si l'accuracy en validation diminue
reduce_lr = ReduceLROnPlateau(monitor='val_loss', 
                              factor=0.2,
                              patience=5, 
                              min_lr=0.001)

# hist = my_new_model.fit(
#                 X_train, 
#                 Y_train,
#                 nb_epoch= epochs,
#                 batch_size=BATCH_SIZE, 
#                 verbose=1, 
#                 validation_data=(X_val, Y_val),
#                 shuffle=True,
#                 callbacks = [reduce_lr])

hist = model.fit(
                X_train, 
                Y_train,
                nb_epoch= epochs,
                batch_size=BATCH_SIZE, 
                verbose=1, 
                validation_data=(X_val, Y_val),
                shuffle=True,
                callbacks = [reduce_lr])

# my_new_model.save('TransferMobileNetV2_12Epochs_doubledenselayer.hdf5')
# my_new_model.save('../Models/TransferResNet50_30Epochs_doubledenselayer.hdf5')
model.save('test.h5')

Train on 28296 samples, validate on 7040 samples
Epoch 1/4
  400/28296 [..............................] - ETA: 2:01:08 - loss: 2.6256 - acc: 0.2200

KeyboardInterrupt: 

<h2>Etude du temps de prédiction pour une image</h2>

In [ ]:
import time
start=time.time()
for i in range(1):
    my_new_model.predict(X_test[i-1:i])

print(time.time()-start)

<h1>Evaluation du modèle</h1>

In [ ]:
my_new_model.evaluate(X_test, Y_test)

<h1>Divers tracés</h1>


<h3>Evolution de l'accuracy train et val en fonction du nombre d'epoch</h3>

In [ ]:
plt.plot(hist.history['acc'])
plt.plot(hist.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

<h3>Evolution de la loss train et val en fonction du nombre d'epoch</h3>

In [ ]:
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

<h1>Chargement du modèle</h1>

In [ ]:
from tensorflow.keras.models import load_model
model = load_model('../Models/TransferMobileNetV2_12Epochs_doubledenselayer.hdf5')

In [ ]:
# Store the model on disk
model.save('tmp.h5')

In [ ]:
# In every test we will clear the session and reload the model to force Learning_Phase values to change.
print('DYNAMIC LEARNING_PHASE')
K.clear_session()
model = load_model('tmp.h5')
# This accuracy should match exactly the one of the validation set on the last iteration.
print(model.evaluate_generator(ImageDataGenerator().flow(x, y, seed=42)))

In [ ]:
print('STATIC LEARNING_PHASE = 0')
K.clear_session()
K.set_learning_phase(0)
model = load_model('tmp.h5')
# Again the accuracy should match the above.
print(model.evaluate_generator(ImageDataGenerator().flow(x, y, seed=42)))

In [ ]:
print('STATIC LEARNING_PHASE = 1')
K.clear_session()
K.set_learning_phase(1)
model = load_model('tmp.h5')
# The accuracy will be close to the one of the training set on the last iteration.
print(model.evaluate_generator(ImageDataGenerator().flow(x, y, seed=42)))